In [ ]:
%matplotlib inline 

from imports import *

print('cuda' if torch.cuda.is_available() else 'cpu')

# Code to Solve the Compressed-Sensing Magnetic Resonance Imaging (CS-MRI) Reconstruction Problem

## Problem formulation

$\boldsymbol{y} = \boldsymbol{M} \odot (\boldsymbol{F} \boldsymbol{X} \boldsymbol{F} + \boldsymbol{\varepsilon}) , \hspace{2ex} \boldsymbol{\varepsilon} \sim \mathcal{N} (0, \sigma^2 \boldsymbol{I}), \hspace{2ex} \sigma \geq 0.$

- $\boldsymbol{X} \in \mathbb{R}^{n \times n}$: unknown image to recover 
- $\boldsymbol{F} \in \mathbb{C}^{n \times n}$: DFT matrix (https://en.wikipedia.org/wiki/DFT_matrix) 
- $\boldsymbol{M} \in \{0 ,1\}^{n \times n}$: 0-1 mask that denotes which Fourier coefficients are observed 
- $\boldsymbol{\varepsilon} \in \mathbb{R}^{m}$: noise
- $ \sigma \in \mathbb{R}^+$: noise level 
- $ \boldsymbol{y} \in \mathbb{C}^{m}$: observed measurements (Fourier coefficients) 
- $ \odot : \mathbb{C}^{n_1 \times n_2} \times \mathbb{C}^{n_1 \times n_2} \mapsto \mathbb{C}^{n_1 \times n_2}$: denotes the Kronecker product (https://en.wikipedia.org/wiki/Kronecker_product)


## Loss function, gradients

### Loss function:

$\ell ({\boldsymbol{X}}) = \frac{1}{2m} \| \boldsymbol{M} \odot (\boldsymbol{y} - \boldsymbol{F} \boldsymbol{X} \boldsymbol{F}) \|_2^2$

### Loss function as a finite sum:

$\ell (\boldsymbol{X}) := \frac{1}{m} \sum_{\{ i, j | y_{i, j} \not= 0 \}} \ell_{i,j}(\boldsymbol{X}) = \frac{1}{m}\sum_{\{ i, j | y_{i, j} \not= 0 \}} \frac{1}{2}(y_i - \boldsymbol{f}_i^\top \boldsymbol{X} \boldsymbol{f}_j)^2$

- $\boldsymbol{f}_i \in \mathbb{C}^m$: the $i^{\text{th}}$ column of the DFT matrix $\boldsymbol{F}$.


### Full gradient:

$\nabla \ell ({\boldsymbol{X}}) = \frac{1}{m}\boldsymbol{F}^\dagger \boldsymbol{M} \odot(\boldsymbol{y} - \boldsymbol{F} \boldsymbol{X} \boldsymbol{F} ) \boldsymbol{F}^\dagger$

### Stochastic gradient:

$\nabla \ell_{i,j}({\boldsymbol{X}}) = (y_i - \boldsymbol{f}_j^\dagger \boldsymbol{X}^\dagger \boldsymbol{f}_i^\ast) \boldsymbol{f}_j^\dagger\boldsymbol{f}^\ast $

- $\dagger : \mathbb{C}^{n_1 \times n_2} \mapsto \mathbb{C}^{n_2 \times n_1} $: Hermitian transpose
- $\ast : \mathbb{C}^{n_1 \times n_2} \mapsto \mathbb{C}^{n_1 \times n_2} $: Complex conjugate

In [ ]:
network_type = 'DnCNN'
denoiser = Denoiser(net=eval(network_type)(17), 
                    experiment_name='exp1_flickr30k_' + network_type, 
                    data=False, sigma=30, batch_size=10)

In [ ]:
original = process_img('./data/Set12/13.jpg')['original']
noisy = process_img('./data/Set12/13.jpg')['noisy']

psnr_display('./data/Set12/13.jpg', original, "Original")

In [ ]:
psnr_display('./data/Set12/13.jpg', noisy, "Original")

In [ ]:
T1 = list(np.arange(0, 45, 5)[1:])
T2 = list(np.arange(0, 45, 5)[1:])
eta = list(np.arange(0.01, 0.1, 0.01))
# batch = list(range(10, 500, 100))
iters = list(product(T1, T2, eta))


'''
during cross-validation, only consider testing when T1 >= T2
'''
# pruned = []

# for each in iters:
#     if each[0] >= each[1]:
#         pruned.append(each)

In [ ]:
def f(iters):
    return peak_signal_noise_ratio(original, svrg(img_path='./data/Set12/13.jpg', 
                                                  denoiser=denoiser.net.cpu(),
                                                  eta=iters[2], T1=iters[0], T2=iters[1], 
                                                  batch_size=200)[0])

with MP.Pool(processes=5) as pool:
    results = list(tqdm.tqdm(pool.imap(f, pruned), total=len(pruned)))

In [ ]:
max(results), pruned[results.index(max(results))]

In [ ]:
svrg_out, svrg_time, svrg_psnr, zs = svrg(img_path='./data/Set12/13.jpg', 
                                          denoiser='nlm', 
                                          eta=0.08, T1=20, T2=10, batch_size=200)
psnr_display('./data/Set12/13.jpg', svrg_out, "SVRG")

In [ ]:
svrg_out, svrg_time, svrg_psnr, zs = svrg(img_path='./data/Set12/13.jpg', 
                                       denoiser=denoiser.net, 
                                       eta=500, T1=1, T2=5000, batch_size=200)
psnr_display('./data/Set12/13.jpg', svrg_out, "SVRG")

In [ ]:
gif(zs)